In [1]:
## Memorializing this as the easy way to lookup accounts


In [1]:
"""
Main Entry Point
"""
from pathlib import Path
from typing import List
import argparse
from datetime import datetime

import pandas as pd # type: ignore

import src.fetch_data # type: ignore
import src.core_transform # type: ignore
import cdutils.pkey_sqlite # type: ignore
import cdutils.hhnbr # type: ignore


data = src.fetch_data.fetch_data()

# # # Core transformation pipeline
raw_data = src.core_transform.main_pipeline(data)

# Raw data with pkey appended
raw_data = cdutils.pkey_sqlite.add_pkey(raw_data)
raw_data = cdutils.pkey_sqlite.add_ownership_key(raw_data)
raw_data = cdutils.pkey_sqlite.add_address_key(raw_data)

In [2]:
househldacct = data['househldacct'].copy()
raw_data = cdutils.hhnbr.add_hh_nbr(raw_data, househldacct)

In [3]:
raw_data

,effdate,acctnbr,ownersortname,product,noteopenamt,ratetypcd,mjaccttypcd,currmiaccttypcd,curracctstatcd,noteintrate,...,Net Balance,Net Available,Net Collateral Reserve,Total Exposure,orig_ttl_loan_amt,portfolio_key,ownership_key,address_key,householdnbr,datelastmaint
0,2025-07-01,150200005,MONEYGRAM PAYMENT SYSTEMS INC,Treasurer's Check,0.0,None,BKCK,TRCK,ACT,0.000000,...,1.594822e+09,0.0,0.0,1.594822e+09,0.0,41514,43261.0,44260.0,187340.0,2020-02-26 23:28:03
1,2025-07-01,151197194,CHAVES HOLDINGS INC,Business Elite Money Market,0.0,VAR,CK,CK30,ACT,0.024500,...,3.216325e+07,0.0,0.0,3.216325e+07,0.0,3843,NaN,NaN,NaN,NaT
2,2025-07-01,150936915,REDBROOK APARTMENTS LLC,Commercial Mortgages,65500000.0,VAR,CML,CM40,ACT,0.065777,...,2.900000e+07,0.0,0.0,2.900000e+07,65500000.0,57112,59661.0,61147.0,193906.0,2023-09-22 08:01:38
3,2025-07-01,151058057,NBPIV SARATOGA LLC,Commercial Mortgages,26000000.0,FIX,CML,CM40,ACT,0.063500,...,2.600000e+07,0.0,0.0,2.600000e+07,26000000.0,79220,83546.0,85431.0,288455.0,2024-06-11 12:02:04
4,2025-07-01,150887241,CITY OF TAUNTON,ICS Shadow - Muni - MMDA/SAV,0.0,None,CK,CK42,ACT,0.000000,...,2.596269e+07,0.0,0.0,2.596269e+07,0.0,4183,33668.0,33967.0,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89772,2025-07-01,28163508,"COSMO, JOEL A.",Prime Time Checking,0.0,VAR,CK,CK06,ACT,0.000300,...,-1.156100e+03,0.0,0.0,-1.156100e+03,0.0,40465,41934.0,42975.0,112816.0,2020-02-26 23:27:59
89773,2025-07-01,27047636,BRISTOL COUNTY SAVINGS BANK,Payoff Overages,0.0,None,BKCK,BTC2,ACT,0.000000,...,-2.098590e+03,0.0,0.0,-2.098590e+03,0.0,1221,1352.0,41645.0,255924.0,2022-09-27 22:46:27
89774,2025-07-01,27039064,BRISTOL COUNTY SAVINGS BANK,Escrow Checks Processing,0.0,None,BKCK,BTC3,ACT,0.000000,...,-7.325560e+04,0.0,0.0,-7.325560e+04,0.0,1221,1352.0,41577.0,255924.0,2022-09-27 22:46:27
89775,2025-07-01,27020568,BRISTOL COUNTY SAVINGS BANK,PDO Checks - OLD,0.0,None,BKCK,BTC1,ACT,0.000000,...,-2.041281e+05,0.0,0.0,-2.041281e+05,0.0,1221,1352.0,41437.0,255924.0,2022-09-27 22:46:27


In [4]:
import cdutils.loans.calculations

In [5]:
loan_category_df = cdutils.loans.calculations.categorize_loans(raw_data)

In [6]:
loan_category_df = loan_category_df[['acctnbr','Category']].copy()

In [31]:
df = pd.merge(raw_data, loan_category_df, on='acctnbr', how='left')

In [32]:
df

,effdate,acctnbr,ownersortname,product,noteopenamt,ratetypcd,mjaccttypcd,currmiaccttypcd,curracctstatcd,noteintrate,...,Net Available,Net Collateral Reserve,Total Exposure,orig_ttl_loan_amt,portfolio_key,ownership_key,address_key,householdnbr,datelastmaint,Category
0,2025-07-01,150200005,MONEYGRAM PAYMENT SYSTEMS INC,Treasurer's Check,0.0,None,BKCK,TRCK,ACT,0.000000,...,0.0,0.0,1.594822e+09,0.0,41514,43261.0,44260.0,187340.0,2020-02-26 23:28:03,NaN
1,2025-07-01,151197194,CHAVES HOLDINGS INC,Business Elite Money Market,0.0,VAR,CK,CK30,ACT,0.024500,...,0.0,0.0,3.216325e+07,0.0,3843,NaN,NaN,NaN,NaT,NaN
2,2025-07-01,150936915,REDBROOK APARTMENTS LLC,Commercial Mortgages,65500000.0,VAR,CML,CM40,ACT,0.065777,...,0.0,0.0,2.900000e+07,65500000.0,57112,59661.0,61147.0,193906.0,2023-09-22 08:01:38,CRE
3,2025-07-01,151058057,NBPIV SARATOGA LLC,Commercial Mortgages,26000000.0,FIX,CML,CM40,ACT,0.063500,...,0.0,0.0,2.600000e+07,26000000.0,79220,83546.0,85431.0,288455.0,2024-06-11 12:02:04,CRE
4,2025-07-01,150887241,CITY OF TAUNTON,ICS Shadow - Muni - MMDA/SAV,0.0,None,CK,CK42,ACT,0.000000,...,0.0,0.0,2.596269e+07,0.0,4183,33668.0,33967.0,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89772,2025-07-01,28163508,"COSMO, JOEL A.",Prime Time Checking,0.0,VAR,CK,CK06,ACT,0.000300,...,0.0,0.0,-1.156100e+03,0.0,40465,41934.0,42975.0,112816.0,2020-02-26 23:27:59,NaN
89773,2025-07-01,27047636,BRISTOL COUNTY SAVINGS BANK,Payoff Overages,0.0,None,BKCK,BTC2,ACT,0.000000,...,0.0,0.0,-2.098590e+03,0.0,1221,1352.0,41645.0,255924.0,2022-09-27 22:46:27,NaN
89774,2025-07-01,27039064,BRISTOL COUNTY SAVINGS BANK,Escrow Checks Processing,0.0,None,BKCK,BTC3,ACT,0.000000,...,0.0,0.0,-7.325560e+04,0.0,1221,1352.0,41577.0,255924.0,2022-09-27 22:46:27,NaN
89775,2025-07-01,27020568,BRISTOL COUNTY SAVINGS BANK,PDO Checks - OLD,0.0,None,BKCK,BTC1,ACT,0.000000,...,0.0,0.0,-2.041281e+05,0.0,1221,1352.0,41437.0,255924.0,2022-09-27 22:46:27,NaN


In [33]:
import cdutils.inactive_date

df = cdutils.inactive_date.append_inactive_date(df)

In [34]:
df

,effdate,acctnbr,ownersortname,product,noteopenamt,ratetypcd,mjaccttypcd,currmiaccttypcd,curracctstatcd,noteintrate,...,Net Collateral Reserve,Total Exposure,orig_ttl_loan_amt,portfolio_key,ownership_key,address_key,householdnbr,datelastmaint,Category,inactivedate
0,2025-07-01,150200005,MONEYGRAM PAYMENT SYSTEMS INC,Treasurer's Check,0.0,None,BKCK,TRCK,ACT,0.000000,...,0.0,1.594822e+09,0.0,41514,43261.0,44260.0,187340.0,2020-02-26 23:28:03,NaN,NaT
1,2025-07-01,151197194,CHAVES HOLDINGS INC,Business Elite Money Market,0.0,VAR,CK,CK30,ACT,0.024500,...,0.0,3.216325e+07,0.0,3843,NaN,NaN,NaN,NaT,NaN,NaT
2,2025-07-01,150936915,REDBROOK APARTMENTS LLC,Commercial Mortgages,65500000.0,VAR,CML,CM40,ACT,0.065777,...,0.0,2.900000e+07,65500000.0,57112,59661.0,61147.0,193906.0,2023-09-22 08:01:38,CRE,NaT
3,2025-07-01,151058057,NBPIV SARATOGA LLC,Commercial Mortgages,26000000.0,FIX,CML,CM40,ACT,0.063500,...,0.0,2.600000e+07,26000000.0,79220,83546.0,85431.0,288455.0,2024-06-11 12:02:04,CRE,NaT
4,2025-07-01,150887241,CITY OF TAUNTON,ICS Shadow - Muni - MMDA/SAV,0.0,None,CK,CK42,ACT,0.000000,...,0.0,2.596269e+07,0.0,4183,33668.0,33967.0,NaN,NaT,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89772,2025-07-01,28163508,"COSMO, JOEL A.",Prime Time Checking,0.0,VAR,CK,CK06,ACT,0.000300,...,0.0,-1.156100e+03,0.0,40465,41934.0,42975.0,112816.0,2020-02-26 23:27:59,NaN,NaT
89773,2025-07-01,27047636,BRISTOL COUNTY SAVINGS BANK,Payoff Overages,0.0,None,BKCK,BTC2,ACT,0.000000,...,0.0,-2.098590e+03,0.0,1221,1352.0,41645.0,255924.0,2022-09-27 22:46:27,NaN,NaT
89774,2025-07-01,27039064,BRISTOL COUNTY SAVINGS BANK,Escrow Checks Processing,0.0,None,BKCK,BTC3,ACT,0.000000,...,0.0,-7.325560e+04,0.0,1221,1352.0,41577.0,255924.0,2022-09-27 22:46:27,NaN,NaT
89775,2025-07-01,27020568,BRISTOL COUNTY SAVINGS BANK,PDO Checks - OLD,0.0,None,BKCK,BTC1,ACT,0.000000,...,0.0,-2.041281e+05,0.0,1221,1352.0,41437.0,255924.0,2022-09-27 22:46:27,NaN,NaT


In [11]:
ASSET_DIR = Path('./assets')

In [12]:
import os

In [13]:
excel_files = [f for f in os.listdir(ASSET_DIR) if f.endswith(('.xls','.xlsx'))]

In [14]:
assert len(excel_files) == 1, f"Expect 1 excel file at {ASSET_DIR}, found {len(excel_files)}."

In [15]:
file_path = os.path.join(ASSET_DIR, excel_files[0])


In [64]:
# Needs to be adjusted to sheet name
worksheet = pd.read_excel(file_path, sheet_name='2025 #1 Semi-Annual Report', header=1)

In [65]:
worksheet

,ORIGINATION DATE OF LOAN,BORROWER'S NAME,STREET ADDRESS,CITY,ST,ZIP,ORIGINAL LOAN COMMITTED AMOUNT,06/30/25 OUTSTANDING PRINCIPAL BALANCE,INCOME TRACT LEVEL,LOAN INTEREST RATE,ORGANIZATION DATE OF THE BORROWER,MINORITY OWNED,NEW JOB CREATION,VETERAN,HIDE ACCOUNT#
0,2025-01-07,"Sweet Metal Finishing, Inc.",28 John Williams Street,Attleboro,MA,2703.0,100000.0,NaN,Middle,0.1200,2005-09-29,Unknown,Unknown,Unknown,151135748.0
1,2025-01-08,"CEI Newman, LLC",991 Newman Avenue,Seekonk,MA,2771.0,401250.0,NaN,Upper,0.0700,2025-01-06,Unknown,Unknown,Unknown,151134948.0
2,2025-01-09,"Mr Cesspool, Inc.",690 Main Street,Acushnet,MA,2743.0,72700.0,NaN,Middle,0.1100,2013-12-05,Unknown,Unknown,Unknown,151135821.0
3,2025-01-15,Fall River Hat Company,847 Pleasant Street,Fall River,MA,2723.0,35000.0,NaN,Low,0.0950,2012-12-04,Unknown,Unknown,Unknown,151136697.0
4,2025-01-15,Fall River Hat Company,847 Pleasant Street,Fall River,MA,2723.0,115000.0,NaN,Low,0.0750,2012-12-04,Unknown,Unknown,Unknown,151136796.0
5,2025-01-16,"Brunello, Inc.",125 High Street,Boston,MA,2110.0,100000.0,NaN,Upper,0.1050,2013-12-16,Unknown,Unknown,Unknown,151139641.0
6,2025-01-24,"Stephen & Co Salon & Spa, Inc.",8 Champion Terrace,Dartmouth,MA,2747.0,10000.0,NaN,Middle,0.0750,2015-01-01,Unknown,Unknown,Unknown,151141406.0
7,2025-01-27,"Capital Strategic Solutions, LLC",43 Broad Street,Hudson,MA,1749.0,84000.0,NaN,Middle,0.1150,2019-01-17,Unknown,Unknown,Unknown,151143569.0
8,2025-01-28,"Russell Frade Enterprise, Inc.",93 Braley Road,East Freetown,MA,2717.0,40000.0,NaN,Upper,0.1100,2022-06-01,Unknown,Unknown,Unknown,151143197.0
9,2025-02-07,"Tia Maria's Café, Inc.",42 North Water Street,New Bedford,MA,2740.0,35000.0,NaN,Low,0.0850,2015-04-16,Unknown,Unknown,Unknown,151136944.0


In [66]:
worksheet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 15 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   ORIGINATION DATE OF LOAN                 56 non-null     datetime64[ns]
 1   BORROWER'S NAME                          56 non-null     object        
 2   STREET ADDRESS                           56 non-null     object        
 3   CITY                                     56 non-null     object        
 4   ST                                       56 non-null     object        
 5   ZIP                                      56 non-null     float64       
 6   ORIGINAL LOAN COMMITTED AMOUNT           57 non-null     float64       
 7   06/30/25 OUTSTANDING PRINCIPAL BALANCE   0 non-null      float64       
 8   INCOME TRACT LEVEL                       56 non-null     object        
 9   LOAN INTEREST RATE                       56 n

In [67]:
worksheet = worksheet.rename(columns={
    'HIDE ACCOUNT#':'acctnbr'
}).copy()

In [68]:
df_slice = df[['acctnbr','notebal']].copy()

In [69]:
df_slice

,acctnbr,notebal
0,150200005,1.594822e+09
1,151197194,3.216325e+07
2,150936915,6.550000e+07
3,151058057,2.600000e+07
4,150887241,2.596269e+07
...,...,...
89772,28163508,-1.156100e+03
89773,27047636,-2.098590e+03
89774,27039064,-7.325560e+04
89775,27020568,-2.041281e+05


In [70]:
worksheet = worksheet.dropna(subset=['acctnbr'])

In [71]:
worksheet_schema = {
    'acctnbr':'int'
}
worksheet = cdutils.input_cleansing.enforce_schema(worksheet, worksheet_schema)

worksheet_schema = {
    'acctnbr':'str'
}
worksheet = cdutils.input_cleansing.enforce_schema(worksheet, worksheet_schema)

df_slice_schema = {
    'acctnbr':'str'
}
df_slice = cdutils.input_cleansing.enforce_schema(df_slice, df_slice_schema)

In [72]:
result = pd.merge(worksheet, df_slice, on='acctnbr', how='left')

In [73]:
result

,ORIGINATION DATE OF LOAN,BORROWER'S NAME,STREET ADDRESS,CITY,ST,ZIP,ORIGINAL LOAN COMMITTED AMOUNT,06/30/25 OUTSTANDING PRINCIPAL BALANCE,INCOME TRACT LEVEL,LOAN INTEREST RATE,ORGANIZATION DATE OF THE BORROWER,MINORITY OWNED,NEW JOB CREATION,VETERAN,acctnbr,notebal
0,2025-01-07,"Sweet Metal Finishing, Inc.",28 John Williams Street,Attleboro,MA,2703.0,100000.0,NaN,Middle,0.1200,2005-09-29,Unknown,Unknown,Unknown,151135748,30041.74
1,2025-01-08,"CEI Newman, LLC",991 Newman Avenue,Seekonk,MA,2771.0,401250.0,NaN,Upper,0.0700,2025-01-06,Unknown,Unknown,Unknown,151134948,397315.28
2,2025-01-09,"Mr Cesspool, Inc.",690 Main Street,Acushnet,MA,2743.0,72700.0,NaN,Middle,0.1100,2013-12-05,Unknown,Unknown,Unknown,151135821,49978.96
3,2025-01-15,Fall River Hat Company,847 Pleasant Street,Fall River,MA,2723.0,35000.0,NaN,Low,0.0950,2012-12-04,Unknown,Unknown,Unknown,151136697,35000.00
4,2025-01-15,Fall River Hat Company,847 Pleasant Street,Fall River,MA,2723.0,115000.0,NaN,Low,0.0750,2012-12-04,Unknown,Unknown,Unknown,151136796,106968.78
5,2025-01-16,"Brunello, Inc.",125 High Street,Boston,MA,2110.0,100000.0,NaN,Upper,0.1050,2013-12-16,Unknown,Unknown,Unknown,151139641,1000.00
6,2025-01-24,"Stephen & Co Salon & Spa, Inc.",8 Champion Terrace,Dartmouth,MA,2747.0,10000.0,NaN,Middle,0.0750,2015-01-01,Unknown,Unknown,Unknown,151141406,2211.73
7,2025-01-27,"Capital Strategic Solutions, LLC",43 Broad Street,Hudson,MA,1749.0,84000.0,NaN,Middle,0.1150,2019-01-17,Unknown,Unknown,Unknown,151143569,83847.51
8,2025-01-28,"Russell Frade Enterprise, Inc.",93 Braley Road,East Freetown,MA,2717.0,40000.0,NaN,Upper,0.1100,2022-06-01,Unknown,Unknown,Unknown,151143197,27000.00
9,2025-02-07,"Tia Maria's Café, Inc.",42 North Water Street,New Bedford,MA,2740.0,35000.0,NaN,Low,0.0850,2015-04-16,Unknown,Unknown,Unknown,151136944,19953.43
